In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
import tensorflow as tf
import torchsnooper

In [21]:
class rbm(nn.Module):
    def __init__(self,shape,para):
        super(rbm,self).__init__()
        self.para=para
        self.W=torch.from_numpy(np.random.normal(scale=1.0/np.sqrt(shape[0])
                                                 ,size=(shape[0],shape[1]))).double().cuda()
        self.bv=torch.zeros(shape[0],device='cuda',dtype=torch.double)
        self.bh=torch.zeros(shape[1],device='cuda',dtype=torch.double)
        print("rbm init complete")
#     @torchsnooper.snoop()
    def forward(self,v):
        v=torch.tensor(v,device='cuda')
#         v=v.view(v.shape[1],v.shape[0])
        self.h=self.sample(torch.sigmoid(torch.matmul(v,self.W)+self.bh))
        v_sample=self.sample(torch.sigmoid(torch.matmul(self.h,
                                                        torch.transpose(self.W,0,1)+self.bv)))
        h_sample=self.sample(torch.sigmoid(torch.matmul(v_sample,self.W)+
                                          self.bh))
        lr=self.para["learning_rate"]/torch.tensor(self.para["batch_size"],dtype=torch.double)
        self.W=self.W+lr*(torch.matmul(torch.transpose(v,0,1),self.h)-
                                        torch.matmul(torch.transpose(v_sample,0,1),h_sample))
        self.bv=self.bv+lr*(torch.mean(v-v_sample,0))
        self.bh=self.bh+lr*(torch.mean(self.h-h_sample,0))
        self.error=torch.sum(torch.pow(v-v_sample,2))
        return self.error
        
        
    def sample(self,probs):
        x=probs+torch.from_numpy(np.random.normal(
                                                    size=probs.shape)).cuda()      
        return x//1.0